## Principios de prompting
- **Principio 1: Instrucciones claras y específicas**
- **Principle 2: Hay que darle al modelo tiempo para pensar**

In [1]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.getenv('OPENAI_API_KEY')

#### Función get_completion
Usaremos el `gpt-3.5-turbo` model y el [chat completions endpoint](https://platform.openai.com/docs/guides/chat). Una api que nos permite hacer preguntas y obtener respuestas.

Esta función va a ayudar a hacer prompts y mirar las salidas

In [2]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

### Tácticas

#### Táctica 1: Utilizar delimitadores para indicar claramente partes distintas de la entrada
- Los delimitadores pueden ser cualquier cosa como:  ```, """, < >, `<tag> </tag>`, `:`

In [3]:
texto = f"""
Debes expresar lo que deseas que un modelo haga \ 
proporcionando instrucciones que sean lo más claras y \ 
específicas posible. \ 
Esto guiará al modelo hacia la salida deseada, \ 
y reducirá las posibilidades de recibir respuestas irrelevantes \ 
o incorrectas. No confundas escribir una \ 
indicación clara con escribir una indicación corta. \ 
En muchos casos, las indicaciones más largas proporcionan más claridad \ 
y contexto al modelo, lo que puede llevar a \ 
salidas más detalladas y relevantes.
"""

prompt = f"""
Resume el texto delimitado por triples comillas \ 
en una sola oración.
```{texto}```
"""
respuesta = get_completion(prompt)
print(respuesta)


Debes proporcionar instrucciones claras y específicas para guiar al modelo hacia la salida deseada y evitar respuestas irrelevantes o incorrectas, aunque las indicaciones más largas pueden proporcionar más claridad y contexto para obtener salidas más detalladas y relevantes.


#### Táctica 2: Solicitar una respuesta estructurada
- JSON, HTML

In [4]:
prompt = f"""
Genera una lista de tres títulos ficticios de libros, \ 
junto con sus autores y géneros. 
Proporciónalos en formato JSON con las siguientes claves: 
book_id, título, autor, género.
"""
respuesta = get_completion(prompt)
print(respuesta)

{
  "book_id": 1,
  "título": "El misterio del faro",
  "autor": "Laura García",
  "género": "Misterio"
},
{
  "book_id": 2,
  "título": "El secreto de la montaña",
  "autor": "Carlos Martínez",
  "género": "Aventura"
},
{
  "book_id": 3,
  "título": "La magia de los sueños",
  "autor": "María López",
  "género": "Fantasía"
}


#### Táctica 3: Solicitarle al modelo que verifique que se satisfagan condiciones

In [5]:
texto_1 = f"""
¡Preparar una taza de té es fácil! En primer lugar, necesitas \ 
poner a hervir un poco de agua. Mientras eso sucede, \ 
toma una taza y coloca una bolsita de té en ella. Una vez que el agua esté \ 
suficientemente caliente, simplemente viértela sobre la bolsita de té. \ 
Deja que repose un poco para que el té pueda infusionarse. Después de unos \ 
minutos, saca la bolsita de té. Si lo deseas, puedes añadir azúcar o leche al gusto. \ 
¡Y eso es todo! Tienes una deliciosa \ 
taza de té para disfrutar.
"""

texto_2 = f"""
El sol brilla intensamente hoy, y los pájaros están \
cantando. Es un hermoso día para dar un \ 
paseo por el parque. Las flores están floreciendo, y los \ 
árboles se mecen suavemente con la brisa. La gente \ 
está afuera, disfrutando del hermoso clima. \ 
Algunos están teniendo picnics, mientras que otros están jugando \ 
juegos o simplemente relajándose en el césped. Es un \ 
día perfecto para pasar tiempo al aire libre y apreciar la \ 
belleza de la naturaleza.
"""

prompt = f"""
Se te proporcionará un texto delimitado por triples comillas. 
Si contiene una secuencia de instrucciones, \ 
reescríbelas en el siguiente formato:

Paso 1 - ...
Paso 2 - …
…
Paso N - …

Si el texto no contiene una secuencia de instrucciones, \ 
simplemente escribe "No se proporcionaron pasos".

\"\"\"{texto_1}\"\"\"
"""

print("Completado para Texto 1:")
print(get_completion(prompt))

print("\nCompletado para Texto 2:")
print(get_completion(prompt.replace(texto_1, texto_2)))


Completado para Texto 1:
Paso 1 - Poner a hervir un poco de agua.
Paso 2 - Tomar una taza y colocar una bolsita de té en ella.
Paso 3 - Verter el agua caliente sobre la bolsita de té.
Paso 4 - Dejar que el té repose para que se infusione.
Paso 5 - Sacar la bolsita de té.
Paso 6 - Opcionalmente, añadir azúcar o leche al gusto.
Paso 7 - Disfrutar de una deliciosa taza de té.

Completado para Texto 2:
No se proporcionaron pasos.


#### Táctica 4: "Few-shot" prompting (pocas muestras)

In [6]:
prompt = f"""
Tu tarea es responder de manera coherente.

<niño>: Enséñame sobre la paciencia.

<abuelo>: El río que talla el valle más profundo fluye desde un manantial modesto; \ 
la sinfonía más grandiosa tiene su origen en una sola nota; \ 
el tapiz más intrincado comienza con un solo hilo.

<niño>: Enséñame sobre la resiliencia.
"""
print(get_completion(prompt))


<abuelo>: La resiliencia es como un árbol que se dobla con el viento, pero nunca se quiebra. Es la capacidad de enfrentar los desafíos y superar las adversidades. Al igual que el árbol, podemos aprender a adaptarnos y encontrar fuerza en medio de las dificultades. La resiliencia nos enseña a levantarnos una y otra vez, incluso cuando pareciera que todo está en contra nuestra. Es una cualidad valiosa que nos permite crecer y transformarnos en personas más fuertes y sabias.


### Principio 2: Darle al modelo tiempo para pensar

#### Táctica 1: Especificar los pasos requeridos para una tarea

In [11]:
texto = f"""
En un encantador pueblo, los hermanos Jack y Jill se embarcaron en \ 
una misión para recoger agua de un pozo en la cima de una colina. \ 
Mientras subían, cantando con alegría, la desgracia \ 
los golpeó: Jack tropezó con una piedra y rodó \ 
colina abajo, seguido por Jill. \ 
A pesar de algunos golpes, la pareja regresó a casa y recibió \ 
abrazos reconfortantes. A pesar del percance, \ 
sus espíritus aventureros no se apagaron, y continuaron explorando con alegría.
"""

prompt = f"""
Tu tarea es realizar las siguientes acciones: 
1 - Resume el siguiente texto delimitado por 
  <> en 1 oración.
2 - Traduce el resumen al francés.
3 - Enumera cada nombre en el resumen en francés.
4 - Genera un objeto JSON que contenga las 
  siguientes claves: french_summary, num_names.

Utiliza el siguiente formato:
Texto: <texto a resumir>
Resumen: <resumen>
Traducción: <traducción del resumen>
Nombres: <lista de nombres en el resumen en francés>
Salida JSON: <json con el resumen y num_names>

Texto: <{texto}>
"""
respuesta = get_completion(prompt)
print(respuesta)


Resumen: Los hermanos Jack y Jill se embarcaron en una misión para recoger agua de un pozo en la cima de una colina, pero sufrieron un accidente al tropezar y rodar colina abajo, aunque regresaron a casa y continuaron explorando con alegría.
Traducción: Les frères Jack et Jill se sont lancés dans une mission pour collecter de l'eau d'un puits au sommet d'une colline, mais ont eu un accident en trébuchant et en roulant en bas de la colline, bien qu'ils soient rentrés chez eux et aient continué à explorer avec joie.
Nombres: Jack, Jill
Salida JSON: {"french_summary": "Les frères Jack et Jill se sont lancés dans une mission pour collecter de l'eau d'un puits au sommet d'une colline, mais ont eu un accident en trébuchant et en roulant en bas de la colline, bien qu'ils soient rentrés chez eux et aient continué à explorer avec joie.", "num_names": 2}


#### Táctica 2: Instruir al modelo a proponer su propia solución antes de apurarse a una conclusión

In [12]:
prompt = f"""
Determina si la solución del estudiante es correcta o no.

Pregunta:
Estoy construyendo una instalación de energía solar y necesito \
ayuda para calcular los aspectos financieros. 
- El costo del terreno es de $100 por pie cuadrado.
- Puedo comprar paneles solares por $250 por pie cuadrado.
- Negocié un contrato de mantenimiento que costará \ 
un flat de $100,000 al año, y un adicional de $10 por pie \
cuadrado.
¿Cuál es el costo total para el primer año de operaciones 
como función del número de pies cuadrados?

Solución del estudiante:
Sea x el tamaño de la instalación en pies cuadrados.
Costos:
1. Costo del terreno: 100x
2. Costo de los paneles solares: 250x
3. Costo de mantenimiento: 100,000 + 100x
Costo total: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""
respuesta = get_completion(prompt)
print(respuesta)

La solución del estudiante es correcta. El costo total para el primer año de operaciones como función del número de pies cuadrados es de 450x + 100,000.


Ojo que la solución del estudiante en verdad está mal. La manera de ayudar al modelo es hacer que el modelo proponga su propia solución antes de apurarse a una conclusión.

In [17]:
prompt = f"""
Su tarea es determinar si la solución del estudiante \
es correcta o no.
Para resolver el problema haga lo siguiente:
- Primero, encuentre su propia solución al problema.
- Luego compara tu solución con la solución del estudiante \
y evalue si la solución del estudiante es correcta o no.
No decida si la solución del estudiante es correcta hasta que
usted mismo haya resuelto el problema.

Utilice el siguiente formato para la respuesta:
Pregunta:
```
pregunta aquí
```
La solución del estudiante:
```
la solución del estudiante aquí
```
Solución real:
```
pasos para encontrar la solución y su solución aquí
```
¿La solución del estudiante es la misma que la solución recién calculada?:
```
sí o no
```
Calificación del estudiante:
```
correcto o incorrecto
```

Pregunta:
```
Estoy construyendo una instalación de energía solar y necesito ayuda \
resolver las finanzas.
- El terreno cuesta $100 / pie cuadrado
- Puedo comprar paneles solares por $250 / pie cuadrado
- Negocié un contrato de mantenimiento que costará \
100.000 dólares fijos al año y 10 dólares adicionales por pie cuadrado \
¿Cuál es el costo total del primer año de operaciones
en función del número de pies cuadrados?
```
La solución del estudiante:
```
Sea x el tamaño de la instalación en pies cuadrados.
Costos:
1. Costo del terreno: 100x
2. Costo del panel solar: 250x
3. Costo de mantenimiento: 100.000 + 100x
Costo total: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
Solución real:
"""
respuesta = get_completion(prompt)
print(respuesta)

Para encontrar el costo total del primer año de operaciones en función del número de pies cuadrados, debemos sumar el costo del terreno, el costo de los paneles solares y el costo de mantenimiento.

1. Costo del terreno: $100 x pies cuadrados
2. Costo de los paneles solares: $250 x pies cuadrados
3. Costo de mantenimiento: $100,000 + $10 x pies cuadrados

Por lo tanto, el costo total del primer año de operaciones en función del número de pies cuadrados es:

Costo total = Costo del terreno + Costo de los paneles solares + Costo de mantenimiento
Costo total = $100 x + $250 x + $100,000 + $10 x
Costo total = $360 x + $100,000

¿La solución del estudiante es la misma que la solución recién calculada?
No

Calificación del estudiante:
Incorrecto


## Limitaciones del modelo: Alucinaciones
- Boie es una compañia real, pero el nombre del producto no es real.

In [18]:
prompt = f"""
Cuentame sobre el cepillo AeroGlide UltraSlim Smart de Boie
"""
response = get_completion(prompt)
print(response)

El cepillo AeroGlide UltraSlim Smart de Boie es un cepillo de dientes innovador y de alta tecnología que ofrece una experiencia de cepillado superior. Diseñado con cerdas de polímero suave y flexibles, este cepillo es suave con las encías y los dientes, pero al mismo tiempo efectivo para eliminar la placa y las manchas.

Una de las características más destacadas de este cepillo es su diseño ultra delgado, lo que lo hace perfecto para llegar a todas las áreas de la boca, incluyendo los espacios interdentales de difícil acceso. Además, su mango ergonómico y antideslizante proporciona un agarre cómodo y seguro durante el cepillado.

El cepillo AeroGlide UltraSlim Smart también cuenta con tecnología inteligente. Viene con un temporizador de 2 minutos que te ayuda a asegurarte de cepillarte durante el tiempo recomendado por los dentistas. Además, tiene un sensor de presión que te alerta si estás aplicando demasiada fuerza al cepillar, lo que puede dañar las encías y el esmalte dental.

Otra